In [2]:
import numpy as np
from keras.layers import Dense, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pandas as pd
import keras.backend as K

In [5]:
data = pd.read_csv("train_foo.csv")   #Loading our csv file 
dataset = np.array(data)              #Converting CSV file into np.array
np.random.shuffle(dataset) #Shuffling our dataset to create even distribution of our datasets
#Storing our dataset into two variables X and Y
X = dataset
Y = dataset
X = X[:, 1:2501]   #X contains all the pixels values,i.e, 50x50=2500
Y = Y[:, 0]

In [6]:
#Splitting our dataset into train and test
X_train = X[0:12000, :]
X_train = X_train / 255.
X_test = X[12000:13201, :]
X_test = X_test / 255.

In [7]:
Y = Y.reshape(Y.shape[0], 1)
Y_train = Y[0:12000, :]
Y_train = Y_train.T
Y_test = Y[12000:13201, :]
Y_test = Y_test.T

In [8]:
print("Number of training examples = " + str(X_train.shape[0]))
print("Number of test examples = " + str(X_test.shape[0]))
print("X_train shape : " + str(X_train.shape))
print("X_test shape : " + str(X_test.shape))
print("Y_train shape : " + str(Y_train.shape))
print("Y_test shape : " + str(Y_test.shape))

Number of training examples = 12000
Number of test examples = 1199
X_train shape : (12000, 2500)
X_test shape : (1199, 2500)
Y_train shape : (1, 12000)
Y_test shape : (1, 1199)


In [10]:
image_x = 50
image_y = 50
#One hot encoding on train and test of Y
train_y = np_utils.to_categorical(Y_train)    
test_y = np_utils.to_categorical(Y_test)
train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)
print("X_train shape: ", str(X_train.shape))
print("X_test shape: ", str(X_test.shape))
print("train_y shape: ", str(train_y.shape))
print("test_y shape: ", str(test_y.shape))

X_train shape:  (12000, 50, 50, 1)
X_test shape:  (1199, 50, 50, 1)
train_y shape:  (12000, 12)
test_y shape:  (1199, 12)


In [13]:
def keras_model(image_x, image_y):
    num_of_classes = 12
    model = Sequential()
    model.add(Conv2D(32, kernel_size = (5, 5), input_shape = (image_x, image_y, 1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'))
    model.add(Conv2D(64, kernel_size = (5, 5), input_shape = (image_x, image_y, 1), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (5, 5), strides = (5, 5), padding = 'same'))
    model.add(Flatten())
    model.add(Dense(1024, activation = 'relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    filepath = "hand_gestures.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
    callbacks_list = [checkpoint1]
    
    return model, callbacks_list

In [14]:
model, callbacks_list = keras_model(image_x, image_y)
model.fit(X_train, train_y, validation_data = (X_test, test_y), epochs = 10, batch_size = 64,
         callbacks = callbacks_list)
scores = model.evaluate(X_test, test_y, verbose = 0)
print("CNN Error: %.2f%%" % (100 - scores[1] * 100))
print_summary(model)
model.save("hand_gestures.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 12000 samples, validate on 1199 samples
Epoch 1/10
12000/12000 [==============================] - 92s 8ms/step - loss: 0.1652 - acc: 0.9497 - val_loss: 0.0161 - val_acc: 0.9975

Epoch 00001: val_acc improved from -inf to 0.99750, saving model to hand_gestures.h5
Epoch 2/10
12000/12000 [==============================] - 83s 7ms/step - loss: 0.0086 - acc: 0.9972 - val_loss: 0.0144 - val_acc: 0.9975

Epoch 00002: val_acc did not improve from 0.99750
Epoch 3/10
12000/12000 [==============================] - 81s 7ms/step - loss: 0.0032 - acc: 0.9990 - val_loss: 0.0088 - val_acc: 0.9983

Epoch 00003: val_acc improved from 0.99750 to 0.99833, saving model to hand_gestures.h5
Epoch 4/10
12000/12000 [==============================] - 81s 7ms/step - loss: 